In [7]:
import time
import random
from gtts import gTTS
from io import BytesIO
from pydub import AudioSegment
from pydub.playback import play

def generate_bingo_call():
    # 定义BINGO的号码范围
    ranges = {'B': range(1, 16), 'I': range(16, 31), 'N': range(31, 46), 'G': range(46, 61), 'O': range(61, 76)}
    # 随机选择一个列
    column = random.choice(list(ranges.keys()))
    # 从选择的列中随机选择一个号码
    number = random.choice(list(ranges[column]))
    return f"{column} {number}"

# 进行5次叫号
for i in range(50):
    # 生成宾果叫号
    call = generate_bingo_call()
    print(i + 1, ':  ', call, '\n')

    # 生成语音
    tts = gTTS(call, lang='zh-cn')
    mp3_fp = BytesIO()
    tts.write_to_fp(mp3_fp)
    mp3_fp.seek(0)

    # 使用pydub加载MP3数据
    audio = AudioSegment.from_file(mp3_fp, format="mp3")

    # 播放音频
    play(audio)

    
    # sleep 1 to 1.5 seconds
    sleep_time = random.uniform(2, 3)
    time.sleep(sleep_time)

1 :   B 2 

2 :   B 14 

3 :   O 72 



KeyboardInterrupt: 

In [ ]:
class VGGA_ST_stage(nn.Module):
    def __init__(self, VGG19, ST):
        super(VGGA_ST_stage, self).__init__()
        self.vgg_transform = transforms.Compose([transforms.Resize((224,224))])
        self.vgg19_backbone = nn.Sequential(*list(VGG19.children())[1:-7])
        ST.flatten = nn.ReLU()
        self.ST_backbone = ST
        self.MP = nn.MaxPool1d(4)
        self.vgg_linear3 = nn.Linear(6272,6272)
        nn.init.xavier_uniform(self.vgg_linear3.weight)
        self.vgg_linear4 = nn.Linear(6272,2048)
        nn.init.xavier_uniform(self.vgg_linear4.weight)
        self.vgg_linear5 = nn.Linear(2048,128)
        nn.init.xavier_uniform(self.vgg_linear5.weight)
        self.vgg_relu2 = nn.ReLU()
        self.vgg_relu3 = nn.ReLU()
        self.vgg_relu4 = nn.ReLU()
        self.st_linear1 = nn.Linear(1024,1024)
        nn.init.xavier_uniform(self.st_linear1.weight)
        self.st_relu1 = nn.ReLU()
        self.st_linear2 = nn.Linear(1024,256)
        nn.init.xavier_uniform(self.st_linear2.weight)
        self.st_relu2 = nn.ReLU()
        
        self.linear1 = nn.Linear(384,384)
        nn.init.xavier_uniform(self.linear1.weight)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(384,2)
        nn.init.xavier_uniform(self.linear2.weight)
        self.softmax = nn.Softmax(dim=1)

        
#         ViT.fc = nn.Linear(768,768)
#         nn.init.xavier_uniform(model.fc.weight)
#         self.ViT_feature_layer = model
#         self.relu1 = nn.ReLU()
#         self.linear1 = nn.Linear(768,2)
#         nn.init.xavier_uniform(self.linear1.weight)
#         self.softmax = nn.Softmax(dim=1)
    def resize_img_size(self, x):
        for img_idx in range(x.shape[0]):
            if img_idx == 0:
                y = self.vgg_transform(x[img_idx]).unsqueeze(0)
            else:
                y = torch.cat((y, self.vgg_transform(x[img_idx]).unsqueeze(0)),0)
        return y
 
    
    def forward(self, x):
        vgg_output = self.vgg19_backbone(self.resize

In [ ]:
    def forward(self, x):
        vgg_output = self.vgg19_backbone(self.resize_img_size(x))
        vgg_output = vgg_output.view(vgg_output.shape[0],-1)
        vgg_output = self.MP(vgg_output)
        vgg_output = self.vgg_linear3(vgg_output)
        vgg_output = self.vgg_relu2(vgg_output)
        vgg_output = self.vgg_linear4(vgg_output)
        vgg_output = self.vgg_relu3(vgg_output)
        vgg_output = self.vgg_linear5(vgg_output)
        vgg_output = self.vgg_relu4(vgg_output)
        st_output = self.ST_backbone(x)
        st_output = self.st_linear1(st_output)
        st_output = self.st_relu1(st_output)
        st_output = self.st_linear2(st_output)
        st_output = self.st_relu2(st_output)
        
        output = torch.cat((vgg_output, st_output),1)
        output = self.linear1(output)
        output = self.relu1(output)
        output = self.linear2(output)
        output = self.softmax(output)

        return output


In [ ]:
Input Image x
    │
    ├───► [Resize Image Size] ───► [VGG19 Backbone] ──┐
    │                                                 │
    │                                                 ▼
    │                                             [Reshape]
    │                                                 │
    │                                                 ▼
    │                                             [Max Pooling]
    │                                                 │
    │                                                 ▼
    │                                   ┌───────► [Linear vgg_linear3]
    │                                   │             │
    │                                   │             ▼
    │                                   │         [ReLU vgg_relu2]
    │                                   │             │
    │                                   │             ▼
    │                                   │      ┌──► [Linear vgg_linear4]
    │                                   │      │        │
    │                                   │      │        ▼
    │                                   │      │    [ReLU vgg_relu3]
    │                                   │      │        │
    │                                   │      │        ▼
    │                                   │      │ [Linear vgg_linear5]
    │                                   │      │        │
    │                                   │      │        ▼
    │                                   │      │    [ReLU vgg_relu4]
    │                                   │      │
    │                                   │      │
    └─► [ST Backbone] ──────► [Linear st_linear1] ──┘
          │                          │
          │                          ▼
          │                      [ReLU st_relu1]
          │                          │
          │                          ▼
          └────► [Linear st_linear2] ──┘
                                   │
                                   ▼
                               [ReLU st_relu2]
                                   │
                                   ▼
                         ┌─────► [Concatenate]
                         │            │
                         │            ▼
                         │       [Linear linear1]
                         │            │
                         │            ▼
                         │       [ReLU relu1]
                         │            │
                         │            ▼
                         │       [Linear linear2]
                         │            │
                         │            ▼
                         └───► [Softmax]
